期中提案僅針對最新的dataset (2023-04)進行資料清理與EDA

https://d37ci6vzurychx.cloudfront.net/trip-data/fhvhv_tripdata_2023-04.parquet

pip install pyarrow

# Import Packages and Dataset

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')
%matplotlib inline

In [2]:
fhvhv_202304 = pd.read_parquet('../fhvhv_tripdata_2023-04.parquet', engine='pyarrow')

In [3]:
fhvhv_202304.head()

,hvfhs_license_num,dispatching_base_num,originating_base_num,request_datetime,on_scene_datetime,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,trip_miles,...,sales_tax,congestion_surcharge,airport_fee,tips,driver_pay,shared_request_flag,shared_match_flag,access_a_ride_flag,wav_request_flag,wav_match_flag
0,HV0003,B03404,B03404,2023-04-01 00:01:28,2023-04-01 00:02:41,2023-04-01 00:03:03,2023-04-01 00:15:42,174,126,5.10,...,1.82,0.00,0.0,0.00,13.83,N,N,,N,N
1,HV0003,B03404,B03404,2023-04-01 00:39:40,2023-04-01 00:41:20,2023-04-01 00:43:42,2023-04-01 01:01:08,211,163,3.11,...,3.56,2.75,0.0,9.52,24.84,N,N,,N,N
2,HV0003,B03404,B03404,2023-03-31 23:56:31,2023-04-01 00:00:20,2023-04-01 00:01:01,2023-04-01 00:09:48,222,76,2.40,...,1.08,0.00,0.0,0.00,8.10,Y,N,,N,N
3,HV0003,B03404,B03404,2023-04-01 00:12:35,2023-04-01 00:16:04,2023-04-01 00:17:17,2023-04-01 00:19:12,76,124,0.43,...,0.70,0.00,0.0,0.00,5.40,N,N,,N,N
4,HV0003,B03404,B03404,2023-04-01 00:46:23,2023-04-01 00:47:30,2023-04-01 00:47:49,2023-04-01 01:05:23,263,247,4.18,...,1.48,2.75,0.0,0.00,15.65,N,N,,N,N


In [4]:
fhvhv_202304.shape

(19144903, 24)

僅保留Uber (HV0003) 的紀錄

In [5]:
uber_202304 = fhvhv_202304[fhvhv_202304['hvfhs_license_num'] == 'HV0003']
uber_202304.reset_index(drop=True, inplace=True)
uber_202304.shape

(13998413, 24)

# 刪除共乘紀錄

共乘紀錄約10萬筆 (約0.75%)  
Uber的共乘服務舊名為Uber Pool，受疫情影響於2020年3月17暫停  
新版的UberX Share在2022年11月1日上線，官方宣稱共乘的車資有最高20%優惠  
處理所有的資料時要考慮暫停即恢復服務的時間，且無法確定新舊版服務的優惠是否有差異  
若要準確預測價格應分開建模，但專案複雜度會太高

In [6]:
share_records = uber_202304[uber_202304.shared_match_flag == "Y"].shape[0]
all_records = uber_202304.shape[0]
p_share_in_all = round((share_records / all_records * 100), 2)
print(f'Shared records: {share_records}')
print(f'Percentage of shared records in all records: {p_share_in_all}%')

Shared records: 105461
Percentage of shared records in all records: 0.75%


In [7]:
uber_202304 = uber_202304[uber_202304.shared_match_flag == "N"]
uber_202304.reset_index(drop=True, inplace=True)
uber_202304.head()

,hvfhs_license_num,dispatching_base_num,originating_base_num,request_datetime,on_scene_datetime,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,trip_miles,...,sales_tax,congestion_surcharge,airport_fee,tips,driver_pay,shared_request_flag,shared_match_flag,access_a_ride_flag,wav_request_flag,wav_match_flag
0,HV0003,B03404,B03404,2023-04-01 00:01:28,2023-04-01 00:02:41,2023-04-01 00:03:03,2023-04-01 00:15:42,174,126,5.10,...,1.82,0.00,0.0,0.00,13.83,N,N,,N,N
1,HV0003,B03404,B03404,2023-04-01 00:39:40,2023-04-01 00:41:20,2023-04-01 00:43:42,2023-04-01 01:01:08,211,163,3.11,...,3.56,2.75,0.0,9.52,24.84,N,N,,N,N
2,HV0003,B03404,B03404,2023-03-31 23:56:31,2023-04-01 00:00:20,2023-04-01 00:01:01,2023-04-01 00:09:48,222,76,2.40,...,1.08,0.00,0.0,0.00,8.10,Y,N,,N,N
3,HV0003,B03404,B03404,2023-04-01 00:12:35,2023-04-01 00:16:04,2023-04-01 00:17:17,2023-04-01 00:19:12,76,124,0.43,...,0.70,0.00,0.0,0.00,5.40,N,N,,N,N
4,HV0003,B03404,B03404,2023-04-01 00:46:23,2023-04-01 00:47:30,2023-04-01 00:47:49,2023-04-01 01:05:23,263,247,4.18,...,1.48,2.75,0.0,0.00,15.65,N,N,,N,N


# 刪除features

*'hvfhs_license_num'*：已經篩選出Uber，全為'HV0003'；  
*'dispatching_base_num'*, *'originating_base_num'*：基地為哪一個不重要，大部分司機收到需求時應不是從基地出發；  
*'bcf'*, *'sales_tax'*：皆是按固定百分比計算，不須預測；  
*'congestion_surcharge'*：固定時段收費，不須預測；   
*'driver_pay'*：為Uber支付給司機的金額，乘客不需要此資訊；  
*'shared_request_flag'*、*'shared_match_flag'*：刪除共乘紀錄後就不需要了；  
*'access_a_ride_flag'*：不重要的資訊；  
*'wav_request_flag'*、*'wav_match_flag'*：Uber官方宣稱WAV的行程費用與UberX相當，因此除非車資有差異，否則不需要此資訊

In [8]:
uber_202304 = uber_202304.drop(columns=['hvfhs_license_num', 'dispatching_base_num', 'originating_base_num', 'bcf', 'sales_tax', 'congestion_surcharge', 'driver_pay', 'shared_request_flag', 'shared_match_flag', 'access_a_ride_flag', 'wav_request_flag', 'wav_match_flag'], axis=1)
print(f'Num of features after deleting: {uber_202304.shape[1]}')

Num of features after deleting: 12


# 檢查missing value, data types, outlier

In [9]:
uber_202304.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13892952 entries, 0 to 13892951
Data columns (total 12 columns):
 #   Column               Non-Null Count     Dtype         
---  ------               --------------     -----         
 0   request_datetime     13892952 non-null  datetime64[ns]
 1   on_scene_datetime    13892952 non-null  datetime64[ns]
 2   pickup_datetime      13892952 non-null  datetime64[ns]
 3   dropoff_datetime     13892952 non-null  datetime64[ns]
 4   PULocationID         13892952 non-null  int32         
 5   DOLocationID         13892952 non-null  int32         
 6   trip_miles           13892952 non-null  float64       
 7   trip_time            13892952 non-null  int64         
 8   base_passenger_fare  13892952 non-null  float64       
 9   tolls                13892952 non-null  float64       
 10  airport_fee          13892952 non-null  float64       
 11  tips                 13892952 non-null  float64       
dtypes: datetime64[ns](4), float64(5), int32(

In [10]:
uber_202304.loc[:, ['trip_miles', 'trip_time', 'base_passenger_fare', 'tolls', 'airport_fee', 'tips']].describe()

,trip_miles,trip_time,base_passenger_fare,tolls,airport_fee,tips
count,1.389295e+07,1.389295e+07,1.389295e+07,1.389295e+07,1.389295e+07,1.389295e+07
mean,5.012595e+00,1.158220e+03,2.504756e+01,1.121717e+00,1.882889e-01,1.114526e+00
std,5.881791e+00,8.274320e+02,2.114031e+01,3.955256e+00,6.620569e-01,3.156857e+00
min,0.000000e+00,0.000000e+00,-1.162800e+02,0.000000e+00,0.000000e+00,0.000000e+00
25%,1.570000e+00,5.850000e+02,1.187000e+01,0.000000e+00,0.000000e+00,0.000000e+00
50%,2.940000e+00,9.420000e+02,1.895000e+01,0.000000e+00,0.000000e+00,0.000000e+00
75%,6.180000e+00,1.486000e+03,3.058000e+01,0.000000e+00,0.000000e+00,0.000000e+00
max,3.621900e+02,3.085100e+04,1.501240e+03,9.715000e+01,5.000000e+00,1.892000e+02


## 檢查trip_miles

In [11]:
for i in range(75, 101, 1):
    print(f'{i}%: {uber_202304.trip_miles.quantile(i/100)}')

75%: 6.18
76%: 6.41
77%: 6.65
78%: 6.9
79%: 7.18
80%: 7.47
81%: 7.78
82%: 8.1
83%: 8.45
84%: 8.81
85%: 9.19
86%: 9.58
87%: 9.99
88%: 10.43
89%: 10.93
90%: 11.52
91%: 12.21
92%: 13.07
93%: 14.1
94%: 15.31
95%: 16.67
96%: 17.77
97%: 19.03
98%: 21.36
99%: 27.33
100%: 362.19


In [12]:
# threshold_trip_miles = uber_202304.trip_miles.quantile(0.99)
# for_trip_miles_hist = uber_202304[uber_202304.trip_miles < threshold_trip_miles]
# sns.histplot(for_trip_miles_hist.trip_miles, kde=True)

## 檢查trip_time

In [13]:
for j in range(75, 101, 1):
    print(f'{j}%: {uber_202304.trip_time.quantile(j/100)}')

75%: 1486.0
76%: 1517.0
77%: 1550.0
78%: 1584.0
79%: 1620.0
80%: 1657.0
81%: 1697.0
82%: 1738.0
83%: 1783.0
84%: 1829.0
85%: 1880.0
86%: 1933.0
87%: 1991.0
88%: 2054.0
89%: 2122.0
90%: 2198.0
91%: 2281.0
92%: 2375.0
93%: 2483.0
94%: 2608.0
95%: 2757.0
96%: 2941.0
97%: 3180.0
98%: 3521.0
99%: 4105.0
100%: 30851.0


In [14]:
# threshold_trip_time = uber_202304.trip_time.quantile(0.99)
# for_trip_time_hist = uber_202304[uber_202304.trip_time < threshold_trip_time]
# sns.histplot(for_trip_time_hist.trip_time, kde=True)

# 計算等待時間

用'request_datetime'、'on_scene_datetime'、'pickup_datetime'計算乘客和司機等待時間，單位為分鐘，計算至小數點後2位

*'rider_waiting_time'*：叫車到司機抵達的時間差，為乘客等待時間  
*'driver_waiting_time'*：司機抵達到乘客上車的時間差，為司機等待時間

In [15]:
uber_202304['rider_waiting_time'] = uber_202304['on_scene_datetime'] - uber_202304['request_datetime']
uber_202304['rider_waiting_time'] = round(uber_202304['rider_waiting_time'] / np.timedelta64(60, 's'), 2)


uber_202304['driver_waiting_time'] = uber_202304['pickup_datetime'] - uber_202304['on_scene_datetime']
uber_202304['driver_waiting_time'] = round(uber_202304['driver_waiting_time'] / np.timedelta64(60, 's'), 2)

In [16]:
uber_202304[['rider_waiting_time', 'driver_waiting_time']].head()

,rider_waiting_time,driver_waiting_time
0,1.22,0.37
1,1.67,2.37
2,3.82,0.68
3,3.48,1.22
4,1.12,0.32


In [17]:
uber_202304.loc[:, ['rider_waiting_time', 'driver_waiting_time']].describe()

,rider_waiting_time,driver_waiting_time
count,1.389295e+07,1.389295e+07
mean,3.527955e+00,9.900605e-01
std,3.145273e+00,1.544575e+00
min,-1.683000e+02,-2.269000e+02
25%,1.750000e+00,2.700000e-01
50%,2.980000e+00,6.800000e-01
75%,4.770000e+00,1.600000e+00
max,6.610200e+02,1.244030e+03


有等待時間小於0的紀錄，不太合理，檢查有幾筆

In [18]:
print(f'Num of rider waiting time less than 0 mins: {uber_202304[uber_202304.rider_waiting_time < 0].shape[0]}')
print(f'Num of dirver waiting time less than 0 mins: {uber_202304[uber_202304.driver_waiting_time < 0].shape[0]}')

Num of rider waiting time less than 0 mins: 109797
Num of dirver waiting time less than 0 mins: 4470


筆數相當多，不適合刪除，預計將小於0的用0替換

# 費用相關

費用相關的feature：  
● *'tolls'*：紐約的通行費定義複雜，預計與起點終點區域進行比對，來預測通行費  
● *'airport_fee'*：起點或終點其中一個為機場，為2.5美元；兩者皆為機場則為5美元  
● *'tips'*：可用來提供小費參考

比對起點區域、終點區域計算出tolls(通行費)

# Taxi zones資料

In [19]:
taxi_zones = pd.read_csv('organized_taxi_zone_lat_lon.csv')
taxi_zones.head()

,zone,LocationID,borough,lat,lon
0,Newark Airport,1,EWR,40.690243,-74.174270
1,Jamaica Bay,2,Queens,40.612163,-73.817643
2,Allerton/Pelham Gardens,3,Bronx,40.864294,-73.846510
3,Alphabet City,4,Manhattan,40.723853,-73.975209
4,Arden Heights,5,Staten Island,40.556678,-74.189803


In [20]:
uber_202304[uber_202304.PULocationID == 132].airport_fee.value_counts()

2.50    258602
0.00      6231
5.00      2608
1.25       467
Name: airport_fee, dtype: int64

待進行：根據LocationID將資料分為5大區域，EDA可用

待進行：用起點和終點區域比對通行費

# 天氣資料